Import Libraries

In [29]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

Connect to Tally

In [38]:
conn.close()

In [39]:
# Establish connection to Tally via ODBC
conn = pyodbc.connect('DSN=TallyODBC64_9000;DRIVER=Tally ODBC Driver64;PORT=9000;username=admin;password=admin')

# Check if the connection is successful
if conn:
    query = "SELECT $Name FROM ODBCTables"  # Query to fetch data from Tally
    data = pd.read_sql(query, conn)
    # Close the connection
    conn.close()
    # Now you have your data in a pandas DataFrame
    print(data)
else:
    print("Connection to Tally failed.")


C:\Users\User\AppData\Local\Temp\ipykernel_26180\248817509.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql 'SELECT * FROM ODBCTables': ('HY000', 'The driver did not supply an error!')

Load Data from Tally

In [22]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine


# Connect to Tally via ODBC
tally_conn = pyodbc.connect('DSN=TallyODBC64_9000;DRIVER=Tally ODBC DRIVER64;PORT=9000;username=admin;password=admin')

# Query to fetch list of tables (you may need to adjust based on your setup)
tables_query = "SELECT $Name FROM ODBCTables"
tables_df = pd.read_sql(tables_query, tally_conn)

# Connection details for MS SQL Server
server = 'LAPTOP-ACQOGCCP'  # This should be the server instance name in SSMS
database = 'DEKU'  # This should be the database name in SSMS
conn_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes'
ms_sql_conn = pyodbc.connect(conn_string)
cursor = ms_sql_conn.cursor()
# Read each table and store the data in MS SQL Server
table_name = 'Company'
data_query = f"SELECT * FROM {table_name}"
data_df = pd.read_sql(data_query, tally_conn)
data_df = data_df[['$_Name','$_Address1','$_Address2','$StartingFrom']]
#Remove $ sign from column names
data_df.columns = data_df.columns.str.replace('$','')
# Create a table in MS SQL Server using cursor without explicitly specifying column names
cursor.execute(f"CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(MAX)' for col in data_df.columns])})")
# Insert data into the table completely without specifying column names
cursor.executemany(f"INSERT INTO {table_name} VALUES ({', '.join(['?']*len(data_df.columns))})", data_df.values.tolist())
# Commit the transaction
ms_sql_conn.commit()
# Close connections
tally_conn.close()
ms_sql_conn.close()


C:\Users\User\AppData\Local\Temp\ipykernel_2720\3614986851.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql(tables_query, tally_conn)
C:\Users\User\AppData\Local\Temp\ipykernel_2720\3614986851.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_df = pd.read_sql(data_query, tally_conn)


LangChain and LLM Implementation with SQL Server

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("mssql+pyodbc://@LAPTOP-ACQOGCCP/DEKU?trusted_connection=yes&driver=ODBC Driver 17 for SQL Server")
print(db.dialect)
print(db.get_usable_table_names())


mssql
['Company']


In [18]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [17]:
import os 
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [31]:
query = 'What is the company name?'
agent_executor.invoke(
query)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Company
Invoking: `sql_db_schema` with `{'table_names': 'Company'}`



CREATE TABLE [Company] (
	[_Name] VARCHAR(max) COLLATE Latin1_General_CI_AS NULL, 
	[_Address1] VARCHAR(max) COLLATE Latin1_General_CI_AS NULL, 
	[_Address2] VARCHAR(max) COLLATE Latin1_General_CI_AS NULL, 
	[StartingFrom] VARCHAR(max) COLLATE Latin1_General_CI_AS NULL
)

/*
3 rows from Company table:
_Name	_Address1	_Address2	StartingFrom
Shree Hari Enterprise	L-93 GIDC Estete,	Odhav, Ahmedabad-382415	2004-04-01
*/
Invoking: `sql_db_query` with `{'query': 'SELECT _Name FROM Company'}`


[('Shree Hari Enterprise',)]The company name is "Shree Hari Enterprise".

> Finished chain.


{'input': 'What is the company name?',
 'output': 'The company name is "Shree Hari Enterprise".'}